# AutoGen Test Run

## ENV and dependencies

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from datetime import datetime
from typing import Callable, Dict, Literal, Optional, Union

from typing_extensions import Annotated

from autogen import (
    Agent,
    AssistantAgent,
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
    config_list_from_json,
    register_function
)
from autogen.agentchat.contrib import agent_builder
from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor

## LLM Configuration

In [3]:
config_gpt_4o = {
    "model": os.environ.get("GPT_4O_MODEL"),
    "api_type": "azure",
    "api_key": os.environ.get("GPT_4O_API_KEY"),
    "base_url": os.environ.get("GPT_4O_AZURE_ENDPOINT"),
    "api_version": os.environ.get("GPT_4O_API_VERSION"),
}


config_gpt_4_turbo = {
    "model": os.environ.get("GPT_4_TURBO_MODEL"),
    "api_type": "azure",
    "api_key": os.environ.get("GPT_4_TURBO_API_KEY"),
    "base_url": os.environ.get("GPT_4_TURBO_AZURE_ENDPOINT"),
    "api_version": os.environ.get("GPT_4_TURBO_API_VERSION"),
}


config_gpt_35_turbo = {
    "model": os.environ.get("GPT_35_TURBO_MODEL"),
    "api_type": "azure",
    "api_key": os.environ.get("GPT_35_TURBO_API_KEY"),
    "base_url": os.environ.get("GPT_35_TURBO_AZURE_ENDPOINT"),
    "api_version": os.environ.get("GPT_35_TURBO_API_VERSION"),
}

## Scaffolding Test: Single Chatbot Agent

In [24]:
# Test out simplest chatbot agent
from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [config_gpt_35_turbo]},
    code_execution_config=False,
    function_map=None,
    human_input_mode="NEVER",
)

In [25]:
reply = agent.generate_reply(messages=[{"content": "What do you know about finance?", "role": "user"}])
print(reply)

I can help provide information on a variety of finance topics, such as personal finance, investments, budgeting, saving, credit management, and more. I can also assist with explaining financial concepts, offering tips and strategies for financial planning, and answering specific questions you might have about finance. Let me know how I can assist you further!


## Multi-Agent Test: Group Chat Task Decomposition

https://microsoft.github.io/autogen/docs/topics/task_decomposition#approach-2-group-chat

In [11]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Give the task, and send instructions to writer to refine the blog post.",
    code_execution_config=False,
)

planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Given a task, please determine what information is needed to complete the task.
Please note that the information will all be retrieved using Python code. Please only suggest information that can be retrieved using Python code.
""",
    llm_config={"config_list": [config_gpt_35_turbo]},
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config={"config_list": [config_gpt_4_turbo], "cache_seed": None},
    system_message="""Engineer. You write python/bash to retrieve relevant information. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor. 
    Always provide bash code block to install dependency first.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
Write code to print the output so that the others can access the result. When you use code to plot data, save the graph to an image file in the work dir and print out the file path. 
""",
)

writer = AssistantAgent(
    name="Writer",
    llm_config={"config_list": [config_gpt_4_turbo], "cache_seed": None},
    system_message="""Writer. Please write blogs in markdown format (with relevant titles) and put the content in pseudo ```md``` code block. You will write it for a task based on previous chat history. Don't write any code.""",
)

os.makedirs("paper", exist_ok=True)
code_executor = UserProxyAgent(
    name = "Executor",
    system_message = "Executor. Execute the code written by the engineer and report the result.",
    description = "Executor should always be called after the engineer has written the code to be executed.",
    human_input_mode = "ALWAYS",
    code_execution_config = {
        "last_n_messages": 3,
        "executor": LocalCommandLineCodeExecutor(work_dir="paper"),
    },
)

In [12]:
groupchat = GroupChat(
    agents = [user_proxy, planner, engineer, writer, code_executor],
    messages = [],
    max_round = 20,
    speaker_selection_method = "auto",
)

manager = GroupChatManager(
    groupchat=groupchat, 
    llm_config={
        "config_list": [config_gpt_4_turbo], 
        "cache_seed": None
    }
)

In [13]:
task = (
    f"Today is {datetime.now().date()}. Write a blogpost about the stock price performance of Nvidia in the past month."
)
print(task)

Today is 2024-06-07. Write a blogpost about the stock price performance of Nvidia in the past month.


In [14]:
with Cache.disk(cache_seed=41) as cache:
    chat_history = user_proxy.initiate_chat(
        manager,
        message=task,
        cache=cache,
    )

Admin (to chat_manager):

Today is 2024-06-07. Write a blogpost about the stock price performance of Nvidia in the past month.

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

To complete this task, we would need the following information:
1. Historic stock price data of Nvidia for the past month (from 2024-05-07 to 2024-06-07).
2. Closing prices of Nvidia for each trading day in the past month.
3. Any relevant news or events that might have impacted Nvidia's stock price performance during the past month.

--------------------------------------------------------------------------------

Next speaker: Engineer

Engineer (to chat_manager):

To proceed, I will fetch historical stock price data for Nvidia from 2024-05-07 to 2024-06-07 and analyze the changes in closing prices and any related news. To retrieve the required data, we can use an API such as Yahoo Finance, Alpha Vantage, or similar data provide